In [1]:
import pandas as pd
import numpy as np
from PIL import Image
from tensorflow import keras
import os
from os import listdir
from os.path import isfile, join
import cv2
import matplotlib.pyplot as plt

In [2]:
resize_res = 224

def load_image(file):
    try:
        image = Image.open(
            file
        ).convert('LA').resize((resize_res, resize_res))
        arr = np.array(image)
    except:
        print('Problem!')
        arr = np.zeros((resize_res, resize_res, 2))
    return arr

In [3]:
model = keras.models.load_model('Point_Predicted_Cropping')

In [4]:
def crop_image(file, out):
    x_padding = 15
    y_top_padding = 5
    y_bottom_padding = 15
    
    x_shape = plt.imread(file).shape[1]
    y_shape = plt.imread(file).shape[0]
    
    prediction = model.predict(np.array([load_image(file)]))
    
    x = [a * x_shape / resize_res for a in [prediction['x1'][0][0]-x_padding, prediction['x2'][0][0]+x_padding, prediction['x3'][0][0]+x_padding, prediction['x4'][0][0]-x_padding]]
    y = [a * y_shape / resize_res for a in [prediction['y1'][0][0]-y_top_padding, prediction['y2'][0][0]-y_top_padding, prediction['y3'][0][0]+y_bottom_padding, prediction['y4'][0][0]+y_bottom_padding]]
    
    
    def get_position(ind):
        if x[ind] in sorted(x)[:2] and y[ind] in sorted(y)[:2]:
            return 'top-left'
        elif x[ind] in sorted(x)[2:] and y[ind] in sorted(y)[:2]:
            return 'top-right'
        elif x[ind] in sorted(x)[:2] and y[ind] in sorted(y)[2:]:
            return 'bottom-left'
        else:
            return 'bottom-right'

    positions = [get_position(i) for i in range(4)]
    
    top_left_x = x[positions.index('top-left')]
    top_left_y = y[positions.index('top-left')]
    
    top_right_x = x[positions.index('top-right')]
    top_right_y = y[positions.index('top-right')]
    
    bottom_left_x = x[positions.index('bottom-left')]
    bottom_left_y = y[positions.index('bottom-left')]
    
    bottom_right_x = x[positions.index('bottom-right')]
    bottom_right_y = y[positions.index('bottom-right')]
    
    left_x = min(top_left_x,bottom_left_x)
    right_x = max(top_right_x, bottom_right_x)
    bottom_y = max(bottom_left_y, bottom_right_y)
    top_y = min(top_left_y, top_right_y)

    image = cv2.imread(file)[int(top_y):int(bottom_y), int(left_x):int(right_x)]
    cv2.imwrite(out+'/'+file.split('/')[-1], image)

In [5]:
def crop_all(in_dir, out_dir, max_num_to_crop = 1000000000):
    for f in listdir(in_dir)[:max_num_to_crop]:
        try:
            crop_image(in_dir+'/'+f, out_dir)
        except:
            pass

In [6]:
crop_all('PSA/6', 'Cropped_PSA_v1/6', max_num_to_crop=20)